In [ ]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf


def download_data(tickers, start="2005-01-01", end="2024-01-01"):
    """Download adjusted close prices and compute daily returns."""
    data = yf.download(tickers, start=start, end=end, progress=False)["Adj Close"]
    returns = data.pct_change().dropna()
    return returns


def mean_variance_optimization(returns, gamma=1.0):
    """Solve the mean-variance optimization problem for given risk aversion."""
    mu = returns.mean().values
    cov = returns.cov().values
    n = len(mu)
    w = cp.Variable(n)
    objective = cp.Maximize(mu @ w - gamma * cp.quad_form(w, cov))
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve()
    return w.value, float(mu @ w.value), float(w.value.T @ cov @ w.value)


def efficient_frontier(returns, gammas):
    mu = returns.mean().values
    cov = returns.cov().values
    n = len(mu)
    w = cp.Variable(n)
    gamma = cp.Parameter(nonneg=True)
    objective = cp.Maximize(mu @ w - gamma * cp.quad_form(w, cov))
    constraints = [cp.sum(w) == 1, w >= 0]
    prob = cp.Problem(objective, constraints)

    risks, rets, weights = [], [], []
    for g in gammas:
        gamma.value = g
        prob.solve()
        risks.append(float(w.value.T @ cov @ w.value))
        rets.append(float(mu @ w.value))
        weights.append(w.value.copy())
    return np.array(risks), np.array(rets), weights


def simulate_crash(returns, start, end, drop=0.03):
    """Simulate crash by subtracting a drop from returns over a period."""
    crash_returns = returns.copy()
    mask = (returns.index >= start) & (returns.index <= end)
    crash_returns.loc[mask] -= drop
    return crash_returns


def plot_frontier(risks, rets, filename="efficient_frontier.png"):
    plt.figure(figsize=(8, 6))
    plt.plot(risks, rets, "b-")
    plt.xlabel("Variance")
    plt.ylabel("Expected Return")
    plt.title("Efficient Frontier")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()


def plot_weights(weights, tickers, title, filename):
    plt.figure(figsize=(10, 6))
    plt.bar(tickers, weights)
    plt.xticks(rotation=45)
    plt.ylabel("Weight")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()




In [ ]:
tickers = [
    "AAPL",
    "MSFT",
    "GOOGL",
    "AMZN",
    "TSLA",
    "JPM",
    "NVDA",
    "JNJ",
    "V",
    "WMT",
]

returns = download_data(tickers)

# Base allocations for different risk aversion levels
risk_levels = [0.1, 1, 10]
for g in risk_levels:
    w, r, risk = mean_variance_optimization(returns, gamma=g)
    print(f"Risk aversion {g:>4}: Return={r:.4f}, Variance={risk:.6f}")
    plot_weights(w, tickers, f"Allocation gamma={g}", f"weights_gamma_{g}.png")

# Stress scenarios
stress_2008 = simulate_crash(returns, "2008-09-01", "2009-03-01", drop=0.05)
w2008, r2008, risk2008 = mean_variance_optimization(stress_2008)
plot_weights(w2008, tickers, "Allocation under 2008 stress", "weights_2008.png")

stress_2020 = simulate_crash(returns, "2020-02-15", "2020-04-15", drop=0.05)
w2020, r2020, risk2020 = mean_variance_optimization(stress_2020)
plot_weights(w2020, tickers, "Allocation under 2020 stress", "weights_2020.png")

print("Baseline allocation:", np.round(mean_variance_optimization(returns)[0], 3))
print("2008 stress allocation:", np.round(w2008, 3))
print("2020 stress allocation:", np.round(w2020, 3))

# Efficient frontier plot
gammas = np.logspace(-2, 3, 50)
risks, rets, _ = efficient_frontier(returns, gammas)
plot_frontier(risks, rets)
